<a href="https://colab.research.google.com/github/alanland/colab-notebooks/blob/main/privateGPT/privateGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## privateGPT

https://github.com/imartinez/privateGPT


More details can be founded in [![GitHub](https://img.shields.io/github/stars/imartinez/privateGPT?style=social)](https://github.com/imartinez/privateGPT/)

In [ ]:
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


## Installnation

In [ ]:
#@title python 3.8 | 不执行则使用系统版本 python
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2  
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1  
!python --version  
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/VideoCrafter/VideoCrafter &> /dev/null
%cd VideoCrafter 
!export PYTHONPATH=/content/VideoCrafter:$PYTHONPATH 

!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null  
!python3.8 -m pip install pytorch-lightning==1.8.3 omegaconf==2.1.1 einops==0.3.0 transformers==4.25.1
!python3.8 -m pip install opencv-python==4.1.2.30 imageio==2.9.0 imageio-ffmpeg==0.4.2
!python3.8 -m pip install av moviepy
!python3.8 -m pip install -e .

In [1]:
#@title git clone
!git lfs install
!git clone https://github.com/imartinez/privateGPT.git
%cd privateGPT

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'privateGPT'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 286 (delta 0), reused 2 (delta 0), pack-reused 282
Receiving objects: 100% (286/286), 103.03 KiB | 7.92 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/content/privateGPT


In [34]:
#@title checkout
#!git checkout main-local
!git pull

Already up to date.


In [ ]:
#@title install dependencies

# 安装依赖
!pip install -r requirements.txt

# Install dependencies
%cd /content/privateGPT
!pip3 install -r requirements.txt

%cd /content
!rm -rf langchain
!rm -rf llama-cpp-python
!git clone https://github.com/hwchase17/langchain
!git clone https://github.com/abetlen/llama-cpp-python
%cd llama-cpp-python/vendor
!git clone https://github.com/ggerganov/llama.cpp

%cd /content

!pip3 uninstall llama-cpp-python -y
!pip3 install scikit-build
%cd /content/llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 python3 ./setup.py install

%cd /content/langchain
!pip3 uninstall langchain -y
!pip3 install -e .

In [ ]:
#@title Install Koala 7B GGML Q4
!mkdir /content/privateGPT/models/
%cd /content/privateGPT/models/
#!wget https://huggingface.co/TheBloke/koala-7B-GGML/resolve/main/koala-7B.ggmlv3.q4_0.bin
!wget https://huggingface.co/gorborukov/koala-7b-ggml-q4_0/resolve/main/koala-7b-ggml-q4_0.bin

In [31]:
%ls
%ls models/

constants.py  ingest.py*  models/         __pycache__/  requirements.txt
db/           LICENSE     privateGPT.py*  README.md     source_documents/
koala-7b-ggml-q4_0.bin


In [41]:
#@title Modify .env
!cp /content/privateGPT/example.env /content/privateGPT/.env 
!sed -i 's/n_gpu_layers=10/n_gpu_layers=500/' /content/privateGPT/.env 
!sed -i 's/MODEL_TYPE=GPT4All/MODEL_TYPE=LlamaCpp/' /content/privateGPT/.env 
!sed -i 's/ggml-gpt4all-j-v1.3-groovy.bin/koala-7b-ggml-q4_0.bin/' /content/privateGPT/.env 

cp: cannot stat '/content/privateGPT/example.env': No such file or directory


In [ ]:
#@title These were required before (for some filetypes), idk now
!apt install pandoc
!pip3 install unstructured

In [ ]:
#@title Ingest sources
%cd  /content/privateGPT/
!python3 ingest.py

In [ ]:
%cd /content/privateGPT
%ls source_documents
!curl https://ttx-download.oss-cn-hangzhou.aliyuncs.com/temp/%E5%86%B7%E7%9F%A5%E8%AF%86.txt -o source_documents/lengzhishi.txt

In [40]:
#@title Run the chatbot
%ls models
%ls models/koala-7B.ggml.q4_0.bin
%cd /content/privateGPT
!python3 privateGPT.py

koala-7b-ggml-q4_0.bin
ls: cannot access 'models/koala-7B.ggml.q4_0.bin': No such file or directory
/content/privateGPT
Using embedded DuckDB with persistence: data will be stored in: db
Traceback (most recent call last):
  File "/content/privateGPT/privateGPT.py", line 76, in <module>
    main()
  File "/content/privateGPT/privateGPT.py", line 34, in main
    llm = LlamaCpp(model_path=model_path, n_ctx=model_n_ctx, callbacks=callbacks, verbose=False)
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: models/koala-7B.ggml.q4_0.bin. Received error Model path does not exist: models/koala-7B.ggml.q4_0.bin (type=value_error)
Exception ignored in: <function Llama.__del__ at 0x7f6a10dfb2e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_cpp_python-0.1.58-py3.10-linux-x86_64.egg/llama_cpp/llama.py", line 1333,

In [ ]:
# Clear database
!rm -rf /content/privateGPT/db

In [ ]:
# Zip the database if will reuse without recreating vectors
!zip -r /content/db.zip /content/privateGPT/db